In [1]:
from functions import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
stocks = ['gme','spy']

In [3]:
globals()['all_results'] = pd.DataFrame(columns=['TP','FP','TN','FN','accuracy',
                                                 'TPR','TNR','B&H','Trading'])

In [4]:
index = ['a']

for i, r in enumerate(['res', 'res1']):
    globals()[f'res_{i}'] = pd.DataFrame(data = np.arange(1 + 10**(i),
                                                    10+10**(i)).reshape((1,9)),
                                columns=['TP','FP','TN','FN','accuracy',
                                'TPR','TNR','B&H','Trading'], index = index)
pd.concat([globals()[f'res_{i}'] for i in range(0,2)],keys=stocks)

,,TP,FP,TN,FN,accuracy,TPR,TNR,B&H,Trading
gme,a,2,3,4,5,6,7,8,9,10
spy,a,11,12,13,14,15,16,17,18,19


In [5]:
index = ['a']
res = pd.DataFrame(data = np.arange(1,10).reshape((1,9)),columns=['TP','FP','TN','FN','accuracy',
                                'TPR','TNR','B&H','Trading'], index = index)
res1 = pd.DataFrame(data = np.arange(10,19).reshape((1,9)),columns=['TP','FP','TN','FN','accuracy',
                                                 'TPR','TNR','B&H','Trading'],
                   index = index)


pd.concat([res,res1], keys = ['l1','l2'])

,,TP,FP,TN,FN,accuracy,TPR,TNR,B&H,Trading
l1,a,1,2,3,4,5,6,7,8,9
l2,a,10,11,12,13,14,15,16,17,18


In [6]:
for stock in stocks:
    stock_df = f'{stock}_data'
    globals()[stock_df] = data_loading_processing(stock)[2]
    svc_linear = SVC(kernel='linear')
    svc_gaussian = SVC(kernel='rbf')
    svc_poly = SVC(kernel='poly',degree=2)
    decision_tree = DecisionTreeClassifier(criterion='entropy',random_state=0)
    random_forest = RandomForestClassifier(n_estimators=3, max_depth=5,
                                              random_state=1)
    knn_1 = KNeighborsClassifier(n_neighbors=1)
    knn_3 = KNeighborsClassifier(n_neighbors=3)
    knn_5 = KNeighborsClassifier(n_neighbors=5)
    logistic = LogisticRegression()
    lda = LinearDiscriminantAnalysis()
    qda = QuadraticDiscriminantAnalysis()
    models = {'svc_linear': svc_linear, 'svc_gaussian':svc_gaussian,
              'svc_poly':svc_poly, 'decision_tree': decision_tree,
              'random_forest':random_forest, 'qda':qda, 'lda':lda,
              'logistic':logistic, 'knn_1':knn_1, 'knn_3':knn_3, 'knn_5':knn_5}
    results = pd.DataFrame(columns=['TP','FP','TN','FN','accuracy','TPR','TNR'])
    # Formulate predictors and outcomes 
    stock_2020 = globals()[stock_df].query('Year == 2020')
    stock_2021 = globals()[stock_df].query('Year == 2021')
    X_train = stock_2020[['mean_return','volatility']].values
    y_train = stock_2020['Label'].values
    X_test = stock_2021[['mean_return','volatility']].values
    y_test = stock_2021['Label'].values

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    trading_df = pd.DataFrame(columns = ['Trading','B&H'])
    
    for model_name, m in models.items():
        m.fit(X_train,y_train)
        preds = m.predict(X_test)
        results = compile_results(results,m,preds,y_test,model_name = model_name)
        trading_df.loc[model_name,['B&H','Trading']] = \
        [buy_and_hold(globals()[stock_df],2021),
         trade_labels(globals()[stock_df],2021,
                      np.array([1 if p == 'Green' else 0 for p in preds]))]
    # for model_name, m in models.items():
    
    #     results.loc[model_name,'B&H'] = buy_and_hold(globals()[stock_df],2021)
    #     results.loc[model_name,'Trading'] = trade_labels(globals()[stock_df],2021,)
    
    combined_results = pd.concat([results,trading_df],axis=1)
    combined_results = combined_results.sort_values(by=['Trading'],axis=0,
                                                    ascending=False)
    globals()[f'{stock}_results'] = combined_results

In [7]:
pd.concat([globals()[f'{stock}_results'] for stock in stocks], keys = stocks)

TP  FP  TN  FN  accuracy   TPR   TNR   Trading     B&H
gme lda            30   1  17   4      0.90  0.88  0.94  49767.72  807.43
    knn_5          28   1  17   6      0.87  0.82  0.94   45809.4  807.43
    svc_linear     27   1  17   7      0.85  0.79  0.94  45327.32  807.43
    knn_3          29   1  17   5      0.88  0.85  0.94  44865.61  807.43
    logistic       26   1  17   8      0.83  0.76  0.94  44369.46  807.43
    random_forest  26   0  18   8      0.85  0.76  1.00  42120.43  807.43
    decision_tree  28   1  17   6      0.87  0.82  0.94  39126.11  807.43
    knn_1          26   1  17   8      0.83  0.76  0.94   31453.2  807.43
    svc_gaussian   28   1  17   6      0.87  0.82  0.94    9172.4  807.43
    qda            26   2  16   8      0.81  0.76  0.89   8032.21  807.43
    svc_poly       29  11   7   5      0.69  0.85  0.39   2599.65  807.43
spy logistic       19   0  31   2      0.96  0.90  1.00    165.06  127.54
    svc_linear     19   1  30   2      0.94  0.90  0.97    164.84  127.54
    lda            16   0  31   5      0.90  0.76  1.00    163.88  127.54
    knn_5          20   0  31   1      0.98  0.95  1.00    163.66  127.54
    decision_tree  19   4  27   2      0.88  0.90  0.87    162.93  127.54
    random_forest  20   2  29   1      0.94  0.95  0.94    162.87  127.54
    knn_3          21   0  31   0      1.00  1.00  1.00    162.73  127.54
    svc_gaussian   21   1  30   0      0.98  1.00  0.97    162.51  127.54
    knn_1          21   2  29   0      0.96  1.00  0.94    161.95  127.54
    qda            11   0  31  10      0.81  0.52  1.00    158.74  127.54
    svc_poly        0   0  31  21      0.60  0.00  1.00    129.05  127.54